In [ ]:
from mycelia.shared.config import MinerConfig, ValidatorConfig
from mycelia.shared.cycle import get_validator_miner_assignment
from mycelia.miner.train import * 
import bittensor as bt 

subtensor = bt.subtensor('test')

path = "/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/config.yaml"
config = MinerConfig.from_path(path)

PyTorch version 2.9.1 available.


/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rank = 2
# === set up chain worker ===
wallet, subtensor = setup_chain_worker(config)

# === mis ===
device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
tokenizer = get_base_tokenizer(config)

# === set up training ===
expert_manager = ExpertManager(config)

2025-12-13 03:54:00 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_dummy'),)
2025-12-13 03:54:00 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)


In [3]:
from mycelia.shared.modeling.mycelia import * 

resume = False
latest_checkpoint_path = None

logger.info(
    "Get base model for checkpoint",
    group_ids=[config.moe.my_expert_group_id] if config.role == "miner" else None,
    partial=(config.role == "miner"),
)
# get base model
model = get_base_model(
    config,
    expert_manager=expert_manager,
    group_ids=[config.moe.my_expert_group_id] if config.role == "miner" else None,
    partial=(config.role == "miner"),
).to(config.model.device)

logger.info("Load base model for checkpoint")

2025-12-13 03:54:00 [info     ] Get base model for checkpoint [mycelia.shared.modeling.mycelia] group_ids=[1] partial=True
2025-12-13 03:54:00 [info     ] Loading in standard mode for miner [mycelia.shared.modeling.mycelia]


The fast path is not available because one of the required library is not installed. Falling back to torch implementation. To install follow https://github.com/fla-org/flash-linear-attention#installation and https://github.com/Dao-AILab/causal-conv1d


2025-12-13 03:55:34 [info     ] Load base model for checkpoint [mycelia.shared.modeling.mycelia]


In [4]:

# load from checkpoint
if get_nested_attr(config, "ckpt.resume_from_ckpt", False):
    resume, model_version, latest_checkpoint_path = start_model_from(
        rank,
        config,
        primary_ckpt_path=config.ckpt.validator_checkpoint_path,
        secondary_ckpt_path=config.ckpt.checkpoint_path,
    )

2025-12-13 03:55:34 [info     ] rank 2: No checkpoints found in /home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation [mycelia.shared.checkpoint]
2025-12-13 03:55:34 [info     ] rank 2: Latest checkpoint found in /home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/globalver_0_inneropt_0 [mycelia.shared.checkpoint]
2025-12-13 03:55:34 [info     ] primary checkpoint not found [mycelia.shared.checkpoint] primary_ckpt_path=PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/validator_checkpoint')


In [6]:
from mycelia.shared.checkpoint import * 
sd = compile_full_state_dict_from_path(latest_checkpoint_path)

2025-12-13 03:57:17 [info     ] load checkpoint state dict   [mycelia.shared.checkpoint] positional_args=(dict_keys(['model_state_dict', 'loss']),)
2025-12-13 03:57:17 [info     ] loaded checkpoint <OpenFile '/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/globalver_0_inneropt_0/model.pt'> loss nan [mycelia.shared.checkpoint]


In [11]:
len(model.state_dict().keys())

1383

In [10]:
len(sd.keys())

1383

In [2]:
import torch 

a = torch.tensor([1,2,3])
a[1] in a

True

In [ ]:
import hashlib
encrypted_bytes = b'Y\x05\x8f=~-\xb8\xc2\xf5\xbd\xb3\xe8\xa6\xfcQp\xfb7\x92\xdf\x18F\t^\xe5\x9c\r\xac^\xa9\xba\x98 6yE\xe1\x84\xd7\xe9/cP\x9d!\xaeu\xa0x\xeb\r\xa5\x90\x1f\xdd\xf0\x8b\x803\xcds\x94>\xbe\xf4\xb1\xdcd\xf3\xc9.S\xc1\x04OMF\xb53R\x84@\x95\xb0s\x99\xdb\xb4 O\xadjH(\x8c\xbcTu \x00\x00\x00\x00\x00\x00\x00{\xba\xc9\'o\xd2\xda\xee\x8a\xc7\x1f>N\xc13\x8cp\xc1g\xd7u/\xb0~\xed\xff~\xaa\xe7\xc3\x8d\xf5 \x00\x00\x00\x00\x00\x00\x00\x1d0\xee\x0b~\xeb\xb0A\x9d\xb1\x83"\xef\xb1\n3\xff\xa2\r\x0c3J?\n\xc7\xb5\xe2\xf9\xcfy\x90\xef\x8e\x00\x00\x00\x00\x00\x00\x00r\x00\x00\x00\x00\x00\x00\x00\x8397c\xd7\x91K\xe8\xe4\x17\x0f\x96\xdb\xeb\x9c\x9b\xf0\xa7ilK\xc9\x95\x12<O\xe7l?q\xd1F\xbe\xf5\x9c\x1f6q\x9a\x08\xf5\xa5\xf8C\x8d\xcc\x071\x8dF\xf8\xe3sK(\x0b\xff\xc2^3Y\x93\xe6\xf58UI\x99\x82\x88{\xccF \xc41\x08\xd0\xecZ@\xa2\xa3j\x01\tCXi\x8f\xa3\xcf?~\x05\x97]\x1f\xef\xd9(\xff\x0b\xdbKg\xd5?\x18\xe6\x15(\x9f\x97\x0c\x00\x00\x00\x00\x00\x00\x00c\x7f\xa4\xb5\xab\x0e\x84\xf1P\xf0/$\x08\x00\x00\x00\x00\x00\x00\x00AES_GCM_\x91\x81r\x01\x00\x00\x00\x00'
hashlib.sha256(encrypted_bytes).digest()

b's\xc9\xb0\xa7\xb5]\xedLJ\x8aH\x92\xd7\xcd\xbe\x88\xe2\xf4\xe6\x95\xbfX\xddz\x81\x98\xdc\x8e@\xc1\x1dK'